Geração do Dashboard
(http://127.0.0.1:8060/)

In [11]:
import pandas as pd
import math
import numpy as np
from PIL import Image
import os
from collections import Counter
import matplotlib.pyplot as plt
from pathlib import Path

path = Path(os.getcwd())

In [14]:
df = pd.read_csv(path / "Resultado por Conteúdo 1Bim.csv")

In [ ]:
df= pd.concat([df, df2], ignore_index=True)

In [45]:
df.head(10)

,Série,% Acertos,Disciplina,Conteúdo,Descritor,turma A
0,5º ANO A,28.0,Língua Portuguesa - 1100,Gênero textual: lenda,Reconhecer diferentes gêneros textuais.,NaN
1,5º ANO A,28.0,Língua Portuguesa - 1100,Verbos,"Identificar verbos, seus tempos e funções.",NaN
2,5º ANO A,40.0,Língua Portuguesa - 1100,Concordância nominal,Reconhecer o funcionamento de estruturas lingu...,NaN
3,5º ANO A,44.0,Língua Portuguesa - 1100,Tema de um texto,Identificar o tema de um texto.,NaN
4,5º ANO A,48.0,Língua Portuguesa - 1100,Características de gênero textual,Reconhecer os elementos constitutivos de um gê...,NaN
5,5º ANO A,48.0,Língua Portuguesa - 1100,Finalidade de um texto,Identificar a finalidade de textos de diferent...,NaN
6,5º ANO A,48.0,Língua Portuguesa - 1100,Ortografia,Identificar padrões ortográficos na escrita da...,NaN
7,5º ANO B,40.0,Língua Portuguesa - 1100,Tema de um texto,Identificar o tema de um texto.,NaN
8,5º ANO B,48.0,Língua Portuguesa - 1100,Gênero textual: lenda,Reconhecer diferentes gêneros textuais.,NaN
9,5º ANO B,48.0,Língua Portuguesa - 1100,Verbos,"Identificar verbos, seus tempos e funções.",NaN


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd


# # ============================================================
# # Leitura e unificação dos arquivos
# # ============================================================
# path_lp = '/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Lingua Portuguesa.csv'
# path_mat = "/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Matemática .csv"

# df_lp = pd.read_csv(path_lp)
# df_mat = pd.read_csv(path_mat)

# # Normaliza nomes de colunas
# df_lp.columns = df_lp.columns.str.strip()
# df_mat.columns = df_mat.columns.str.strip()

# # Adiciona coluna Disciplina faltante (garantindo nomes consistentes)
# df_lp["Disciplina"] = "Língua Portuguesa"
# df_mat["Disciplina"] = "Matemática"

# Unifica tudo
df = pd.concat([df_lp, df_mat], ignore_index=True)

# Remove headers duplicados
df = df[df["% Acertos"].astype(str) != "% Acertos"]

# Padroniza nomes de colunas
df.columns = df.columns.str.replace("Conteúdo", "Conteudo", regex=False)
df.columns = df.columns.str.replace("Série", "Serie", regex=False)

# Limpa e converte coluna % Acertos
df["% Acertos"] = (
    df["% Acertos"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace("%", "", regex=False)
    .str.strip()
)

df["% Acertos"] = pd.to_numeric(df["% Acertos"], errors="coerce")

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd


# # ============================================================
# # Leitura e unificação dos arquivos
# # ============================================================
# path_lp = '/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Lingua Portuguesa.csv'
# path_mat = "/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Matemática .csv"

# df_lp = pd.read_csv(path_lp)
# df_mat = pd.read_csv(path_mat)

# # Normaliza nomes de colunas
# df_lp.columns = df_lp.columns.str.strip()
# df_mat.columns = df_mat.columns.str.strip()

# # Adiciona coluna Disciplina faltante (garantindo nomes consistentes)
# df_lp["Disciplina"] = "Língua Portuguesa"
# df_mat["Disciplina"] = "Matemática"

# # Unifica tudo
# df = pd.concat([df_lp, df_mat], ignore_index=True)

# Remove headers duplicados
df = df[df["% Acertos"].astype(str) != "% Acertos"]

# Padroniza nomes de colunas
df.columns = df.columns.str.replace("Conteúdo", "Conteudo", regex=False)
df.columns = df.columns.str.replace("Série", "Serie", regex=False)

# Limpa e converte coluna % Acertos
df["% Acertos"] = (
    df["% Acertos"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace("%", "", regex=False)
    .str.strip()
)

df["% Acertos"] = pd.to_numeric(df["% Acertos"], errors="coerce")


# ============================================================
# DASHBOARD
# ============================================================
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Desempenho por Conteúdo"),

    html.Div([
        html.Label("Série / Turma:"),
        dcc.Dropdown(
            id="dropdown-serie",
            options=[{"label": s, "value": s} for s in sorted(df["Serie"].unique())],
            value=sorted(df["Serie"].unique())[0],
            clearable=False
        ),
    ], style={"width": "30%", "display": "inline-block", "margin-right": "30px"}),

    html.Div([
        html.Label("Disciplina:"),
        dcc.Dropdown(
            id="dropdown-disciplina",
            options=[{"label": d, "value": d} for d in sorted(df["Disciplina"].unique())],
            value=sorted(df["Disciplina"].unique())[0],
            clearable=False
        ),
    ], style={"width": "30%", "display": "inline-block"}),

    dcc.Graph(id="grafico-barras")
])


# ============================================================
# CALLBACK
# ============================================================
@app.callback(
    Output("grafico-barras", "figure"),
    Input("dropdown-serie", "value"),
    Input("dropdown-disciplina", "value")
)
def atualizar_grafico(serie, disciplina):

    df_f = df[
        (df["Serie"] == serie) &
        (df["Disciplina"] == disciplina)
    ]

    fig = px.bar(
        df_f,
        x="Conteudo",
        y="% Acertos",
        color="Conteudo",
        text="% Acertos"
    )

    fig.update_layout(
        title=f"% de Acerto por Conteúdo — {disciplina} ({serie})",
        xaxis_title="Conteúdo",
        yaxis_title="% Acertos",
        legend_title="Conteúdos",
        yaxis=dict(range=[0, 100])
    )

    fig.update_traces(texttemplate="%{text:.1f}%", textposition="outside")

    return fig


# ============================================================
# Run
# ============================================================
if __name__ == "__main__":
    app.run(debug=True, port=8060)
